# Блок Импорта

In [ ]:
# Общие библиотеки
import pandas as pd # Библиотека Pandas для работы с табличными данными
import matplotlib.pyplot as plt # библиотека Matplotlib для визуализации
import numpy as np # библиотека Numpy для операций линейной алгебры и прочего

# Модель
from sklearn.linear_model import LogisticRegression  # Логистичекая регрессия от scikit-learn

# Pipeline
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline


# Предварительная обработка
from sklearn.compose import ColumnTransformer # т.н. преобразователь колонок
from sklearn.impute import SimpleImputer# Объект для замены пропущенных значений
from sklearn import preprocessing as prep # Общий модуль предварительной обработки
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler
# Импортируем One-Hot Encoding, нормализацию, Стандартизацию и от scikit-learn
from sklearn.preprocessing import PolynomialFeatures # Объект для генерации полиномиальных признаков от scikit-learn

# Про кросс-валидацию
from sklearn.model_selection import ShuffleSplit # при кросс-валидации случайно перемешиваем данные
from sklearn.model_selection import cross_val_score # Оценка модели через кросс-валидацию

from sklearn.model_selection import train_test_split#  функция разбиения на тренировочную и тестовую выборку
# но в исполнении scikit-learn

# Метрики
from sklearn.metrics import f1_score # f1-мера от scikit-learn
from sklearn.metrics import make_scorer #функция для создания объекта из метрики для использования в кросс-валидации

from sklearn import set_config

In [ ]:
set_config(display="diagram") #чтобы была красивая визуализация

## Ставим optun'у

[Описание Библиотеки](https://optuna.readthedocs.io/en/stable/index.html)

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.3 MB/s eta 0:00:00


In [ ]:
import optuna # библиотека для оптимизации гиперпараметров

# Загружаем данные в Pandas

Загружаем наши данные из файла по прямой ссылке на git-hub

In [ ]:
DF = pd.read_csv('https://raw.githubusercontent.com/dayekb/Basic_ML_Alg/main/cars_moldova_clean.csv', delimiter = ',')
DF

,Make,Model,Year,Style,Distance,Engine_capacity(cm3),Fuel_type,Transmission,Price(euro),km_year
0,Toyota,Prius,2011,Hatchback,195000.0,1800.0,Hybrid,Automatic,7750.0,17727.272727
1,Renault,Grand Scenic,2014,Universal,135000.0,1500.0,Diesel,Manual,8550.0,16875.000000
2,Renault,Laguna,2012,Universal,110000.0,1500.0,Diesel,Manual,6550.0,11000.000000
3,Opel,Astra,2006,Universal,200000.0,1600.0,Metan/Propan,Manual,4100.0,12500.000000
4,Mercedes,Vito,2000,Microvan,300000.0,2200.0,Diesel,Manual,3490.0,13636.363636
...,...,...,...,...,...,...,...,...,...,...
32069,Volkswagen,Passat,2016,Sedan,88000.0,1800.0,Petrol,Automatic,11500.0,14666.666667
32070,Land Rover,Freelander,2002,Crossover,225000.0,1800.0,Metan/Propan,Manual,4400.0,11250.000000
32071,Dacia,Logan Mcv,2015,Universal,89000.0,1500.0,Diesel,Manual,7000.0,12714.285714
32072,Mazda,6,2006,Combi,370000.0,2000.0,Diesel,Manual,4000.0,23125.000000


# Работа с данными

### Посчитаем число численных и категориальных колонок

In [ ]:
cat_columns = []
num_columns = []

for column_name in DF.columns:
    if (DF[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))

print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

categorical columns:	  ['Make', 'Model', 'Style', 'Fuel_type', 'Transmission'] 
 len =  5
numerical columns:	  ['Year', 'Distance', 'Engine_capacity(cm3)', 'Price(euro)', 'km_year'] 
 len =  5


Допустим смотрим задачу **классификации**

In [ ]:
data = DF.drop(columns = ['Transmission']) # в признаках нам не нужен 'Transmission'
target = DF['Transmission'] # поскольку это целевая метка

In [ ]:
cat_columns = cat_columns[:-1] # уберем 'Transmission' из списка категориальных данных

In [ ]:
cat_columns

['Make', 'Model', 'Style', 'Fuel_type']

Для этого воспользуемся объектом `LabelEncoder()` из модуля `preprocessing`

Применение преобразований уже стандартное для нас

* Создаем объект
* обучаем методом `.fit()`
* Смотрим что получилось

In [ ]:
Label = prep.LabelEncoder()
Label.fit(target) # задаем столбец, который хотим преобразовать
Label.classes_ # в аттрибуте .classes_ хранится информация "какой класс как шифруется"

array(['Automatic', 'Manual'], dtype=object)

т.е. `0` это 'Automatic', а `1` это 'Manual'

In [ ]:
Label_encoded = Label.transform(target)  # преобразуем и сохраняем в новую переменную

In [ ]:
Label_encoded # здесь уже только 0 и 1

array([0, 1, 1, ..., 1, 1, 1])

Разобъем Тренировочную выборку на тренировочную и валидационную (не забываем фиксировать сид при разбиении)

In [ ]:
random_state = 42

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(data,Label_encoded,
                                                  test_size = 0.2,
                                                  random_state = random_state)

# Ух

Испытание (`trial`) - это процесс оценки целевой функции.

Предоставляет интерфейсы для получения предложений параметров, управления состоянием испытания и установки/получения определяемых пользователем атрибутов испытания

Ключевые методы про которые стоит знать

* `suggest_categorical()`
Предложить значение для категориального параметра из заданного выбора

* `suggest_discrete_uniform(name, low, high, q)`
Предложить значение для дискретного параметра из равномерного распределения

* `suggest_float(name, low, high, *[, step, log])`
Предложить значение для параметра с плавающей точкой

* `suggest_int(name, low, high[, step, log])`
Предложить значение для целочисленного параметра

* `suggest_loguniform(name, low, high)`
Предложить значение для непрерывного параметра из лог-равномерного распределения

* `suggest_uniform(name, low, high)`
Предложить значение для непрерывного параметра из равномерного распределения


In [ ]:
# -- Принято делать через функцию objective
def objective(trial):
    # -- Можно варьировать вообще всё
    # (a) Допустим выберем разные преобразователи числовых данных:
                                        #↓ название ↓из чего выбираем
    scalers = trial.suggest_categorical("scalers", ['minmax', 'standard', 'robust'])

    # (b) зададим объекты преобразователи
    if scalers == "minmax":
        scaler = MinMaxScaler()
    elif scalers == "standard":
        scaler = StandardScaler()
    else:
        scaler = RobustScaler()

    # -- Допустим мы еще зачем-то хотим проверить поможет ли нам полиномиальные признаки
    # (a) Сгенерим гиперпараметры для полиномиальных признаков
    #                           ↓ название ↓нижний и верхний диапазон изменений
    degree = trial.suggest_int("PF_degree", 1, 3)
                                                #↓ название     ↓из чего выбираем
    interaction = trial.suggest_categorical("PF_interaction", [True, False])

    # (b) зададим объект преобразователь
    PF = PolynomialFeatures(degree=degree,
                        include_bias=False,
                        interaction_only=interaction)

    # -- Создаем Pipeline для числовых данных
    numerical_pipe_poly = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('poly_features', PF),
        ('scaler', scaler),
    ])

    # -- Создаем Pipeline для категориальных данных (но можно его тоже поварьровать)
    categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent', )),
    ('encoder', OneHotEncoder(drop='if_binary', handle_unknown='ignore', sparse_output=False))
    ])

    preprocessors = ColumnTransformer(transformers=[
        ('num', numerical_pipe_poly, num_columns),
        ('cat', categorical_pipe, cat_columns)
    ])

    # -- Готовим модель
    # (a) Посмотрим на константу регуляризации:
                            #↓ название
                            #     ↓нижний и верхний диапазон изменений
                            #               ↓лог-распределение
    C = trial.suggest_float("С", 1e-2, 1e2, log=True)

    # (b) Посмотрим на разные типы регуляризации
                                        #↓ название ↓из чего выбираем
    penalty = trial.suggest_categorical("penalty", ['l1', 'l2'])

    # (c) Собираем модель
    estimator = LogisticRegression(C = C,
                                   penalty = penalty,
                                   random_state=42, max_iter = 10000,solver = 'liblinear')

    # -- Делаем итоговый pipeline
    pipeline = make_pipeline(preprocessors, estimator)

    # -- Указываем метрику по которой будем оптимизировать
    # чтобы, например указать какое-то хитрое усреднение метрик для многоклассовой классификации
    scorer = make_scorer(f1_score, average='macro', zero_division = 0)

    # -- Указываем тип кросс-валидации
    cv=ShuffleSplit(n_splits=5, #на сколько разбиваем
                    random_state=42)

    # -- Оцениваем по кросс-валидации
    score = cross_val_score(pipeline, # какой pipeline
                            # ↓ используются в качестве глобальных переменных
                            X_train, y_train, # на каких данных
                            scoring=scorer, # по какой метрике
                            cv = cv #как проводим кросс-вадлидацию
                            )
    final_score = score.mean() # усредняем по этапам кросс-валидации

    return final_score



## Пытаемся смотреть на то что получилось

Создается объект `study` (можно указать `direction` - увеличиваем ли уменьшаем целевую функцию)

К которому применяется оптимизация целевой функции `objective` описанной нами ранее

In [ ]:
study = optuna.create_study(direction="maximize", # Классификция  поэтому метрику Увеличиваем
                            study_name="Cars Log_Reg"
                            )
study.optimize(objective, # целевая функция которую мы оптимизируем
               n_trials=10, # Генерим 10 раз разные вариации (долго же для примера - вы можете больше указать)
               )


[I 2023-04-24 14:54:48,462] A new study created in memory with name: Cars Log_Reg
[I 2023-04-24 14:54:55,864] Trial 0 finished with value: 0.8524732216297041 and parameters: {'scalers': 'minmax', 'PF_degree': 2, 'PF_interaction': False, 'С': 56.881945194284945, 'penalty': 'l2'}. Best is trial 0 with value: 0.8524732216297041.
[I 2023-04-24 14:54:57,168] Trial 1 finished with value: 0.8405363215342359 and parameters: {'scalers': 'robust', 'PF_degree': 1, 'PF_interaction': False, 'С': 0.01935794552580367, 'penalty': 'l1'}. Best is trial 0 with value: 0.8524732216297041.
[I 2023-04-24 14:55:08,419] Trial 2 finished with value: 0.8503757464297049 and parameters: {'scalers': 'minmax', 'PF_degree': 1, 'PF_interaction': True, 'С': 2.004549595041605, 'penalty': 'l1'}. Best is trial 0 with value: 0.8524732216297041.
[I 2023-04-24 14:55:27,181] Trial 3 finished with value: 0.8531926498988002 and parameters: {'scalers': 'minmax', 'PF_degree': 3, 'PF_interaction': False, 'С': 16.910667936387338, '

In [ ]:
# Результаты
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_PF_degree,params_PF_interaction,params_penalty,params_scalers,params_С,state
0,0,0.852473,2023-04-24 14:54:48.466943,2023-04-24 14:54:55.864239,0 days 00:00:07.397296,2,False,l2,minmax,56.881945,COMPLETE
1,1,0.840536,2023-04-24 14:54:55.866263,2023-04-24 14:54:57.167457,0 days 00:00:01.301194,1,False,l1,robust,0.019358,COMPLETE
2,2,0.850376,2023-04-24 14:54:57.170521,2023-04-24 14:55:08.418487,0 days 00:00:11.247966,1,True,l1,minmax,2.004550,COMPLETE
3,3,0.853193,2023-04-24 14:55:08.421581,2023-04-24 14:55:27.181109,0 days 00:00:18.759528,3,False,l2,minmax,16.910668,COMPLETE
4,4,0.838114,2023-04-24 14:55:27.184872,2023-04-24 14:55:28.454431,0 days 00:00:01.269559,1,False,l1,robust,0.012839,COMPLETE
5,5,0.848816,2023-04-24 14:55:28.456398,2023-04-24 14:55:40.189348,0 days 00:00:11.732950,1,False,l1,minmax,0.240840,COMPLETE
6,6,0.850089,2023-04-24 14:55:40.192305,2023-04-24 14:55:51.066472,0 days 00:00:10.874167,3,True,l1,standard,0.336368,COMPLETE
7,7,0.851164,2023-04-24 14:55:51.069432,2023-04-24 14:55:53.480832,0 days 00:00:02.411400,2,True,l2,standard,1.385237,COMPLETE
8,8,0.840655,2023-04-24 14:55:53.487132,2023-04-24 14:55:57.005802,0 days 00:00:03.518670,2,False,l1,standard,0.021095,COMPLETE
9,9,0.851160,2023-04-24 14:55:57.014356,2023-04-24 14:56:09.420093,0 days 00:00:12.405737,2,True,l1,standard,22.575151,COMPLETE


Лучший результат

In [ ]:
study.best_trial # Смотрим на лучший Pipeline

FrozenTrial(number=3, state=TrialState.COMPLETE, values=[0.8531926498988002], datetime_start=datetime.datetime(2023, 4, 24, 14, 55, 8, 421581), datetime_complete=datetime.datetime(2023, 4, 24, 14, 55, 27, 181109), params={'scalers': 'minmax', 'PF_degree': 3, 'PF_interaction': False, 'С': 16.910667936387338, 'penalty': 'l2'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'scalers': CategoricalDistribution(choices=('minmax', 'standard', 'robust')), 'PF_degree': IntDistribution(high=3, log=False, low=1, step=1), 'PF_interaction': CategoricalDistribution(choices=(True, False)), 'С': FloatDistribution(high=100.0, log=True, low=0.01, step=None), 'penalty': CategoricalDistribution(choices=('l1', 'l2'))}, trial_id=3, value=None)

## Красивая графика

Ребята постарались сколаборировались с `plotly`

История оптимизации

In [ ]:
optuna.visualization.plot_optimization_history(study)

Все оптимизируемые вещи

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

Отдельные визуализации точностей для разных значений гиперпараметров

In [ ]:
optuna.visualization.plot_slice(study)

Не очень понятная, но красивая визуализация пар гиперпараметров (насколько тот или иной гиперпараметр "полезный")

In [ ]:
optuna.visualization.plot_contour(study)

Отдельный график

In [ ]:
optuna.visualization.plot_contour(study,['С','PF_degree'])

## Про важность отдельных гиперпараметров

важность гиперпараметров (чтобы потом повторить и проверить только значимые)

In [ ]:
optuna.visualization.plot_param_importances(study)

# Соберем Лучшую модель

In [ ]:
study.best_params

{'scalers': 'minmax',
 'PF_degree': 3,
 'PF_interaction': False,
 'С': 16.910667936387338,
 'penalty': 'l2'}

Раскидаем вручную гиперпараметры в новый `pipeline`

In [ ]:
scalers = study.best_params['scalers']
if scalers == "minmax":
    scaler = MinMaxScaler()
elif scalers == "standard":
    scaler = StandardScaler()
else:
    scaler = RobustScaler()

PF = PolynomialFeatures(degree=study.best_params['PF_degree'],
                        include_bias=False,
                        interaction_only=study.best_params['PF_interaction'])

numerical_pipe_poly = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('poly_features', PF),
    ('scaler', scaler),
])

categorical_pipe = Pipeline([
('imputer', SimpleImputer(strategy='most_frequent', )),
('encoder', OneHotEncoder(drop='if_binary', handle_unknown='ignore', sparse=False))
])

preprocessors = ColumnTransformer(transformers=[
    ('num', numerical_pipe_poly, num_columns),
    ('cat', categorical_pipe, cat_columns)
])

estimator = LogisticRegression(C = study.best_params['С'],
                               penalty = study.best_params['penalty'],
                                random_state=42, max_iter = 10000,
                               solver = 'liblinear')

# Делаем итоговый pipeline
pipeline = make_pipeline(preprocessors, estimator)

In [ ]:
pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('poly_features',
                                                                   PolynomialFeatures(degree=3,
                                                                                      include_bias=False)),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Year', 'Distance',
                                                   'Engine_capacity(cm3)',
                                                   'Price(euro)', 'km_year']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['Make', 'Model', 'Style',
                                                   'Fuel_type'])])),
                ('logisticregression',
                 LogisticRegression(C=16.910667936387338, max_iter=10000,
                                    random_state=42, solver='liblinear'))])

Оптимизированная модель, ура

Для оптимизации нейронных сетей (или если используется обучение ["по частям"](https://scikit-learn.org/0.15/modules/scaling_strategies.html) ) есть возможность [ранней остановки ](https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/003_efficient_optimization_algorithms.html#pruning-algorithms)